# PLAN
## Initial
#### Safe bet to use Alpha Vantage to develop Model
## Final Step
#### Once ready to do extensive tranning with Multiple Stocks we will need to use SEC EDGAR most likely (need to figure that out) or pay for like a month subscription of Alpha Vantage

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta


# Using FMP
# - This is more accurate and consitent! Yahoo is web scrapping
# - Provides the financial statetement for a broader range then yahoo finance

# Key which is given from my FMP account. (This key is needed to make requests) Maximum of 200 calls/day
API_KEY = "wUvM2M29ZVxHDvK8IRp2P7iyrT8uhQG4"

ticker = "AAPL"

end_date = datetime(2024, 6, 1)
start_date = end_date - timedelta(days=50)

url_hist_stock_price = f"https://financialmodelingprep.com/stable/historical-price-eod/light?symbol={ticker}&from={start_date.date()}&to={end_date.date()}&apikey={API_KEY}"

# Historical Data Request
response_hist_stock_price = requests.get(url_hist_stock_price)

json_response_hist_stock_price = response_hist_stock_price.json()


In [ ]:
closing_price = pd.DataFrame(json_response_hist_stock_price)

# Convert 'date' to datetime type
closing_price['date'] = pd.to_datetime(closing_price['date'])

# Keep only 'date' and 'price' columns
closing_price = closing_price[['date', 'price']]

# Set 'date' as the index
closing_price.set_index('date', inplace=True)

print(closing_price)

In [146]:
# ALPHA VANTAGE EXPERIMENTING (https://www.alphavantage.co/documentation/#)
# gives some insight into the different attributes without sending demo request: https://documentation.alphavantage.co/FundamentalDataDocs/gaap_documentation.html

API_KEY = "FJLURMBXQEXWW9PP"

ticker = "AAPL"

def getInfo(function:str,ticker:str):
    url = f"https://www.alphavantage.co/query?function={function.upper()}&symbol={ticker}&apikey={API_KEY}"
    response = requests.get(url)
    return response.json()

In [147]:
# Income Statement
r_income_statements = getInfo("income_statement",ticker)
r_balance_sheet = getInfo("balance_sheet",ticker)
r_cash_flow = getInfo("cash_flow",ticker)

In [154]:
df_income_statement = pd.DataFrame(r_income_statements["quarterlyReports"])
#print(df_income_statement.columns)

df_income_statement.set_index("fiscalDateEnding", inplace=True)
net_income_str = df_income_statement['netIncome']
net_income = net_income_str.apply(pd.to_numeric, errors='coerce')




In [155]:
df_cash_flow = pd.DataFrame(r_cash_flow["quarterlyReports"])
df_cash_flow.set_index("fiscalDateEnding", inplace=True)

#print(df_cash_flow.columns)

preferedStock_str = df_cash_flow["dividendPayoutPreferredStock"]
preferedStock = preferedStock_str.apply(pd.to_numeric, errors='coerce')
preferedStock = preferedStock.fillna(0)


In [ ]:
df_balance_sheet = pd.DataFrame(r_balance_sheet["quarterlyReports"])
df_balance_sheet.set_index("fiscalDateEnding", inplace=True)

#print(df_balance_sheet.columns)

outstandingShares_str = df_balance_sheet["commonStockSharesOutstanding"]
outstandingShares = outstandingShares_str.apply(pd.to_numeric, errors='coerce')
df_metrics = pd.DataFrame()
df_metrics["EPS"] = (net_income - preferedStock)/ outstandingShares #IMPORTANT DID NOT CONSIDER PREFERED DIVVIDEND HERE

# Find common dates

print(df_metrics)

# Calculate EPS ignoring rows with NaN (optional)




fiscalDateEnding
2025-03-31    15056133000
2024-12-31    15150865000
2024-09-30    15408095000
2024-06-30    15348175000
2024-03-31    15464709000
                 ...     
2006-03-31    24599036000
2005-12-31    24477796000
2005-09-30    24477796000
2005-06-30    24102484000
2005-03-31    23996308000
Name: commonStockSharesOutstanding, Length: 81, dtype: int64
                       EPS
fiscalDateEnding          
2025-03-31        1.645841
2024-12-31        2.397883
2024-09-30        0.956380
2024-06-30        1.397430
2024-03-31        1.528383
...                    ...
2006-03-31        0.016667
2005-12-31        0.023082
2005-09-30        0.017567
2005-06-30        0.013235
2005-03-31        0.012085

[81 rows x 1 columns]
